In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cifar_tools
import tensorflow as tf

In [4]:
learning_rate = 0.001

names, data, labels = cifar_tools.read_data('./cifar-10-batches-py')

names ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
(50000, 3072) (50000,)


In [5]:
x = tf.placeholder(tf.float32, [None, 24*24])
y = tf.placeholder(tf.float32, [None, len(names)])

W1 = tf.Variable(tf.random_normal([5, 5, 1, 64]))
b1 = tf.Variable(tf.random_normal([64]))

W2 = tf.Variable(tf.random_normal([5, 5, 64, 64]))
b2 = tf.Variable(tf.random_normal([64]))

W3 = tf.Variable(tf.random_normal([6*6*64, 1024]))
b3 = tf.Variable(tf.random_normal([1024]))

W_out = tf.Variable(tf.random_normal([1024, len(names)]))
b_out = tf.Variable(tf.random_normal([len(names)]))

In [6]:
def conv_layer(x, W, b):
    conv = tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')
    conv_with_b = tf.nn.bias_add(conv, b)
    conv_out = tf.nn.relu(conv_with_b)
    return conv_out

In [7]:
def maxpool_layer(conv, k=2):
    return tf.nn.max_pool(conv, ksize=[1,k,k,1], strides=[1,k,k,1], padding='SAME')

In [8]:
def model():
    x_reshaped = tf.reshape(x, shape=[-1, 24, 24, 1])
    
    conv_out1 = conv_layer(x_reshaped, W1, b1)
    maxpool_out1 = maxpool_layer(conv_out1)
    norm1 = tf.nn.lrn(maxpool_out1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)
    
    conv_out2 = conv_layer(norm1, W2, b2)
    norm2 = tf.nn.lrn(conv_out2, 4, bias=1.0, alpha=0.001/9.0, beta=0.75)
    maxpool_out2 = maxpool_layer(norm2)
    
    maxpool_reshaped = tf.reshape(maxpool_out2, [-1, W3.get_shape().as_list()[0]])
    local = tf.add(tf.matmul(maxpool_reshaped, W3), b3)
    local_out = tf.nn.relu(local)
    
    out = tf.add(tf.matmul(local_out, W_out), b_out)
    return out

In [9]:
model_op = model()

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model_op, labels=y))

train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

correct_pred = tf.equal(tf.argmax(model_op, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [10]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    onehot_labels = tf.one_hot(labels, len(names), on_value=1., off_value=0., axis=-1)
    onehot_vals = sess.run(onehot_labels)
    batch_size = len(data) // 200
    print('batch size', batch_size)
    for j in range(0, 1000):
        print('EPOCH', j)
        for i in range(0, len(data), batch_size):
            batch_data = data[i:i+batch_size,:]
            batch_onehot_vals = onehot_vals[i:i+batch_size,:]
            _, accuracy_val = sess.run([train_op, accuracy], feed_dict={x: batch_data, y: batch_onehot_vals})
            if i % 1000 == 0:
                print(i, accuracy_val)
        print('DONE WITH EPOCH')

batch size 250
EPOCH 0
0 0.12
1000 0.072
2000 0.092
3000 0.144
4000 0.124
5000 0.112
6000 0.14
7000 0.132
8000 0.164
9000 0.156
10000 0.16
11000 0.148
12000 0.184
13000 0.192
14000 0.184
15000 0.152
16000 0.164
17000 0.188
18000 0.196
19000 0.156
20000 0.196
21000 0.204
22000 0.2
23000 0.232
24000 0.144
25000 0.168
26000 0.224
27000 0.168
28000 0.22
29000 0.2
30000 0.22
31000 0.24
32000 0.26
33000 0.196
34000 0.24
35000 0.216
36000 0.236
37000 0.24
38000 0.248
39000 0.224
40000 0.24
41000 0.208
42000 0.212
43000 0.208
44000 0.2
45000 0.26
46000 0.252
47000 0.192
48000 0.248
49000 0.244
DONE WITH EPOCH
EPOCH 1
0 0.244
1000 0.232
2000 0.228
3000 0.292
4000 0.228
5000 0.236
6000 0.244
7000 0.204
8000 0.236
9000 0.292
10000 0.256
11000 0.248
12000 0.248
13000 0.264
14000 0.224
15000 0.236
16000 0.252
17000 0.28
18000 0.268
19000 0.224
20000 0.232
21000 0.308
22000 0.252
23000 0.272
24000 0.2
25000 0.204
26000 0.296
27000 0.252
28000 0.252
29000 0.248
30000 0.256
31000 0.288
32000 0.268
330

KeyboardInterrupt: 